In [1]:
import numpy as np
%load_ext Cython

In [2]:
# configurations
NDECKS: int = 8
NPLAYERS: int = 4 # affects how many cards you may be able to count
BURNPROB: float = 1.0

In [35]:
"""
We keep our representation of the deck and cards as characters, rather than grouping on BJ value
Importantly, this means our framework remains flexible enough to add in weird house rules such as perfect pairs/flushes
"""

# We use bytestrings as our dtype so we can have a Cython memview over the numpy card stream of C char type
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A'], dtype='S1') # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'], dtype='S1')
deck = np.array([np.char.add(cards, suit) for suit in suits], dtype='S2')
stream = np.repeat(deck, NDECKS)

In [36]:
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(stream)

8.05 µs ± 35.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [44]:
# bad idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards, values))
%timeit np.array([dict_map[x[:1]] for x in stream]) # slicing byte string to get first char
# %timeit np.vectorize(dict_map.get)(stream)

69.2 µs ± 3.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [59]:
%%cython 
import numpy as np
cimport cython 

# Create a packed struct in C representing one card in the stream
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char card
    char suit

""" 
C uses ascii integer representation for characters
Numeric characters in our stream will have an ascii value less than ascii of 'a'
English characters will have an ascii value greater than ascii of 'a'
"""
cdef int ASCII_A = 65 

@cython.boundscheck(False)
@cython.wraparound(False)
def play_stream(card_dtype_struct[:] stream not None):
    


    if stream[0].card == b'2': 
        return 5
    else:
        return 3





In [49]:
stream[:50]

array([b'2H', b'6S', b'JS', b'8S', b'5C', b'6S', b'9D', b'AS', b'JS',
       b'QH', b'7H', b'TH', b'3H', b'6S', b'5S', b'JD', b'4C', b'KH',
       b'QH', b'TC', b'KC', b'2D', b'7S', b'7S', b'5S', b'5D', b'TS',
       b'AS', b'3S', b'4C', b'5S', b'8D', b'JS', b'2C', b'4H', b'AS',
       b'8H', b'TH', b'5H', b'QC', b'3D', b'2H', b'2H', b'9H', b'8H',
       b'4H', b'AC', b'8S', b'7H', b'6S'], dtype='|S2')

In [57]:
play_stream(stream)

5